# Initial Time Series Modeling Code. 
This script contains the first steps for univariate time series modeling. The decomposition of the series and the ACF/PACF plots are created to understand the seres. Auto ARIMA to choose the parameters is run with the option to add in exogenous variables. 


In [ ]:
import pandas as pd #for data analysis/manipulation
import numpy as np
#from azureml import Workspace # connect to the Azure environment 
import pyodbc # connect to the database
import matplotlib.pyplot as plt # plotting package 
import time
import pytz
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import arma_order_select_ic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import datetime
from datetime import timedelta

# Functions 

In [ ]:
def time_fmt(time_entry, original_tz = 'US/Pacific', new_tz = 'US/Eastern'):
    '''Convert the timezone for a timestamp object'''
    input_time = time_entry.replace(tzinfo=pytz.timezone(original_tz))
    conv_time = input_time.astimezone(pytz.timezone(new_tz))
    return conv_time

def mape_calc(actual, predicted):
    '''Caluclate Mean Absoulte Percent Error'''
    act, pred = np.array(actual), np.array(predicted)
    mape = np.mean(np.abs((act - pred)/act)*100)
    return mape

def accuracy_metrics(actual, predicted, pct_error = False):
    '''Calculate MSE, RMSE, MAE and MAPE if requested. '''
    metrics = []
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, predicted)
    mape = mape_calc(actual, predicted)
    
    metrics.append(mse)
    metrics.append(rmse)
    metrics.append(mae)
    metrics.append(mape)

    print('Accuracy Metrics')
    print('MSE: {}'.format(mse))
    print('RMSE: {}'.format(rmse))
    print('MAE: {}'.format(mae))
    print('MAPE: {}'.format(mape))
    
    return metrics

def inverse_difference(data, yhat, interval = 1):
    return yhat + data[-interval]

def predictions_plot(test, predictions, col, days= 14,):
    ''' Create the plot of actuals vs predictions. 
            test is a dataframe iwth Date column. 
            predictions is a list of the predicted values '''
    test_sub = test.head(days)
    test_sub['Predictions'] = predictions
    test_sub = test_sub.reset_index()
    test_sub['Day'] = test_sub['Date'].apply(lambda x: x.weekday_name)
    
    plt.plot(test_sub['Date'], test_sub['Predictions'], color = 'red', label = 'Predictions')
    plt.plot(test_sub['Date'], test[col][:days].values)
    plt.title('Forecast Two Weeks Out')
    plt.xticks(rotation = 45)
    plt.legend(loc = 'best')
    plt.show()
    
    return test_sub


def arima_model(col, p,d,q, lag = False, days=14):
    ''' Run and arima model. The function prints the summary after training, accuracy metrics and plot of predicted values to actuals. '''
    ref = col
    print(ref)
    # call the ARIMA model
    model = ARIMA(train[col].dropna(), (p,d,q))
    res_102 = model.fit()
    print(res_102.summary())

    plt.plot(res_102.resid.values, alpha =0.7)
    plt.hlines(0, xmin = 0, xmax = 800, color = 'r')
    plt.title("ARIMA({},{},{}) ".format(p,d,q)) 
    plt.show()

    results = res_102.fittedvalues.to_frame()

    test14 = test[ref][:days]

    if lag == False:
        accuracy_metrics(train[col], results[0])
        # predict the next 14 days 
        forecast14 = res_102.forecast(days)[0]

        forecast_adj = [0 if x < 0 else x for x in forecast14 ]
        print( 'TEST SET')
        accuracy_metrics(test14, forecast_adj)
        result = predictions_plot(test, forecast_adj, col = ref)
    else:
        
        accuracy_metrics(train[col][days:], results[0])
        forecast14 = res_102.forecast(days)[0]
        
        history = [x for x in train[ref]]
        pred = []

        days = days

        for yhat in forecast14:
            inverted = inverse_difference(history, yhat, days)
            pred.append(inverted)
            days += 1

        pred 

        # adjust predictions to not be negative 
        pred = [0 if x < 0 else x for x in pred ]
        test14 = test[ref][:days]
        print('TEST SET')
        accuracy_metrics(test14, pred)
        predictions = predictions_plot(test, pred, col = ref)
    

In [ ]:
# Read in a dataframe 
df = df 

In [ ]:
# Azure write table to Workspace
#ws = Workspace()
#ws.datasets.add_from_dataframe(dataframe = df, data_type_id = 'GenericCSV' , name = 'file_name', description ='table info')

In [ ]:
# Add Holidays to the table 
cal = calendar()
holidays_list = cal.holidays(start = '2017-03-01', end = '2019-07-01', return_name= True)
holidays = holidays_list.to_frame(name= 'Holiday Name').rename_axis('Date').reset_index()
# holiday name column to the data frame
df = pd.merge(df, holidays,how ='left', on='Date')

In [ ]:
# add the day of the week 
df['Day of the Week'] = df['Date'].apply(lambda x: x.weekday_name)


df['Holiday Flag'] = df['Holiday Name'].notnull().astype(int)

# weekend flag 
df['Weekend Flag'] = [1 if x in ['Saturday', 'Sunday'] else 0 for x in df['Day of the Week']]

# lagssssss
df['lag 1'] = df['target col'].shift()
df['lag 7'] = df['target col'].shift(7)
df['lag 30'] = df['target col'].shift(30)

df['lag 1'] = df['target col'].shift()
df['lag 7'] = df['target col'].shift(7)
df['lag 30'] = df['target col'].shift(30)



### Look at the Decomposition of the series. 

In [ ]:
decomposition_plot(df['target col'], freq=7)

In [ ]:
decomposition_plot(df['target col'], freq=30)

## Plotting the Rolling Mean and Standard Deviation 
Vizual look for stationarity. 

In [ ]:
def plt_rolling(df):
    fig, ax = plt.subplots(3, figsize=(12,9))
    ax[0].plot(df.index, df['target col'], label ='raw data')
    ax[0].plot(df['target col'].rolling(window=7).mean(), label ='rolling mean')
    ax[0].plot(df['target col'].rolling(window=7).std(), label ='rolling std')
    ax[0].legend()
    
    ax[1].plot(df.index, df['z_data'], label ='differenced by 7')
    ax[1].plot(df['z_data'].rolling(window=7).mean(), label ='rolling mean')
    ax[1].plot(df['z_data'].rolling(window=7).std(), label ='rolling std')
    ax[1].legend()
    
    ax[2].plot(df.index, df['zp_data'], label ='raw data')
    ax[2].plot(df['zp_data'].rolling(window=7).mean(), label ='7 lag and differenced')
    ax[2].plot(df['zp_data'].rolling(window=7).std(), label ='rolling std')
    ax[2].legend()
    
    plt.tight_layout()
    fig.autofmt_xdate()
    plt.show()

plt_rolling(df)

## Augmented Dickey-Fuller Test for Stationarity 

In [ ]:
print("Is the data stationary?")
dftest = adfuller(df['target col'], autolag = 'AIC')
print('Test Statistic = {:.3f}'.format(dftest[0]))
print("P-value = {:.3f}".format(dftest[1]))
print("Critical values:")
for k, v in dftest[4].items():
    print("{}% confidence, {} the data is {} stationary".format(100-int(k[:-1]), v, "not" if v < dftest[0] else "" ))


## Ploting the Autocorrlation and Partial Autocorrlation graphs

In [ ]:
fig, ax = plt.subplots(2, figsize=(8,6))
ax[0] = plot_acf(df['target col'].dropna(), ax=ax[0], lags = 20)
ax[1] = plot_pacf(df['target col'].dropna(), ax=ax[1], lags = 20)
plt.show()

### Test/Train split 

In [ ]:
train = df[df['Date'] < '2019-05-01']
test = df[df['Date']>= '2019-05-01']

train = train.set_index('Date')
test = test.set_index('Date')

# Auto ARIMA

In [ ]:
auto_arma = arma_order_select_ic(train['col'], max_ar = 5, max_ma = 5, ic = ['aic', 'bic'])

print("AIC order")
print(auto_arma.aic_min_order)
print('BIC order')
print(auto_arma.bic_min_order)

In [ ]:
model = ARIMA(train['col'], (4,0,4))
res_102 = model.fit()
print(res_102.summary())

plt.plot(res_102.resid.values, alpha =0.7)
plt.hlines(0, xmin = 0, xmax = 800, color = 'r')
plt.title("ARIMA(4,0,4) ") 
plt.show()

In [ ]:
results = res_102.fittedvalues.to_frame()

accuracy_metrics(train['target col'], results[0])

test14 = test['target col'][:14]

# predict the next 14 days 
forecast14 = res_102.forecast(14)[0]

In [ ]:
# adjusting forecasts to not be negative
forecast_adj = [0 if x < 0 else x for x in forecast14 ]
accuracy_metrics(test14, forecast_adj)
result = predictions_plot(test, forecast_adj)

In [ ]:
result['diff'] = result['target col'] - result['Predictions']
result.head()

## Run ARIMA model 

In [ ]:
auto_arma = arma_order_select_ic(train['target col'], max_ar = 5, max_ma = 5, ic = ['aic', 'bic'])

print("AIC order")
print(auto_arma.aic_min_order)
print('BIC order')
print(auto_arma.bic_min_order)

In [ ]:
arima_model('target col', 4,0,4, False)

## Holidays as exogenous variable 

In [ ]:
train = df[df['Date'] < '2019-05-01']
test = df[df['Date']>= '2019-05-01']

train = train.set_index('Date')
test = test.set_index('Date')

In [ ]:
train_holiday = train['Holiday Flag'].values
test_holiday = test['Holiday Flag'].values 

In [ ]:
model = ARIMA(train['target col'], (5,0,4), exog = train_holiday)
res_102 = model.fit()
print(res_102.summary())

plt.plot(res_102.resid.values, alpha =0.7)
plt.hlines(0, xmin = 0, xmax = 800, color = 'r')
plt.title("ARIMA(5,0,4) ") 
plt.show()

In [ ]:
results = res_102.fittedvalues.to_frame()

accuracy_metrics(train['target col'], results[0])

test14 = test['target col'][:14]

test_holidays = test['Holiday Flag'][:14]


forecast14 = res_102.forecast(14, exog = test_holidays)[0]

In [ ]:
forecast_adj = [0 if x < 0 else x for x in forecast14 ]

accuracy_metrics(test14, forecast_adj)
result = predictions_plot(test, forecast_adj, col= 'target col')

In [ ]:
from platform import python_version 
python_version()

In [ ]:
import pandas as pd
pd.__version__